In [ ]:
from fastai.vision.all import *
from fastai import *
from fastcore.all import *

In [ ]:
path=Path()

In [ ]:
data_path=(path/'train_kaggle')
test_path=(path/'test_kaggle/')

In [ ]:
def get_category(filename):
    """Определяет метку по имени файла.
    Возвращает метку""" 
    if ('frankfurter' in filename.name) | ('chili-dog' in filename.name) | ('hotdog' in filename.name):
        y = 1
    else: y = 0
    return y

In [ ]:
db = DataBlock(blocks=[ImageBlock, CategoryBlock], 
                get_items=get_image_files,
                get_y=get_category,
                splitter=RandomSplitter(valid_pct=0),
                item_tfms=RandomResizedCrop(224, min_scale=0.6),
                batch_tfms=aug_transforms)

In [ ]:
dls = db.dataloaders(data_path, bs=32)

In [ ]:
# Обучение модели convnext_tiny с загруженными весами
learn = vision_learner(dls, convnext_tiny, metrics=accuracy)
learn.fine_tune(3)

In [ ]:
test = get_image_files(test_path)
test_dl = learn.dls.test_dl(test)

In [ ]:
learn.get_preds(dl=test_dl, with_decoded=True)

In [ ]:
preds, _, label = learn.get_preds(dl=test_dl, with_decoded=True)

In [ ]:
image_id = []
for i in test:
    image_id=np.append(image_id, i.name)

In [ ]:
with open('subm.csv', 'w') as submissionFile:
    writer = csv.writer(submissionFile)
    writer.writerow(['image_id', 'label'])
    writer.writerows(zip(image_id,np.int32(label)))

In [ ]:
learn.export("hotdog_model.pkl")